In [ ]:
%pip install pandas
%pip install matplotlib

In [ ]:
import pandas as pd

In [ ]:
# If witrina_all_orders_original.csv exists, read it and make a bool variable original
# If not, read the witrina_all_orders_anonymized.csv and make a bool variable original
try:
    df = pd.read_csv('../data/original/witrina_all_orders_original.csv')
    isOriginalDataUsed = True
except:
    df = pd.read_csv('../data/anonymous/witrina_all_orders_anonymized.csv')
    isOriginalDataUsed = False

In [ ]:
df.head()

In [ ]:
# Trim column names
df.columns = df.columns.str.strip()

In [ ]:
# Drop unnecessary columns: password, Billing Address 2, order notes
df = df.drop(['Password', 'Billing Address 2', 'Order note', 'Order notes', 'Order number', 'Weglot Language', 'Modified Date', 'Stripe Charge Captured', 'Order Total'], axis=1)

In [ ]:
# Rename columns
df = df.rename(columns={'Payment Method Title': 'Payment Method', 
                        'record_id': 'Transaction', 
                        'Line items': 'Line Items',
                        'Billing Country': 'Billing Country Code', 
                        'Shipping Country': 'Shipping Country Code'})

# Renaming the first occurrence of "Shipping Method Title"
cols = df.columns.tolist()
cols[cols.index('Shipping Method Title')] = 'Shipping Method 1'
df.columns = cols

# Renaming the second occurrence of "Shipping Method Title"
cols = df.columns.tolist()
cols[cols.index('Shipping Method Title')] = 'Shipping Method 2'
df.columns = cols


In [ ]:
# Define a conditional expression for HRK
condition = (df['Billing Country Code'] == "HR") & (df['Date'] <= "2022-08-12")
exchange_rate = 7.5345

In [ ]:
# Check last rows of the data where Billing Country is Croatia
df[condition].tail(40)

In [ ]:
# Convert HRK to EUR for orders from Croatia that happened before 2022-08-12 (inclusive) in a new column called Total (EUR)
df.loc[(df['Billing Country Code'] == "HR") & (df['Date'] <= "2022-08-12"), 'Total (EUR)'] = (df['Total'] / exchange_rate).round(2)
# Do the same for 'Subtotal' column
df.loc[(df['Billing Country Code'] == "HR") & (df['Date'] <= "2022-08-12"), 'Subtotal (EUR)'] = (df['Subtotal'] / exchange_rate).round(2)

In [ ]:
# Fill NaN values in Total (EUR) column with values from Total column
df['Total (EUR)'].fillna(df['Total'], inplace=True)
# Fill NaN values in Subtotal (EUR) column with values from Subtotal column
df['Subtotal (EUR)'].fillna(df['Subtotal'], inplace=True)

In [ ]:
# Drop Total & Subtotal original columns
df = df.drop(['Total', 'Subtotal'], axis=1)

In [ ]:
# Calculate Subtotal where Subtotal is 0 by subtracting Order Shipping from Total
df.loc[df['Subtotal (EUR)'] == 0, 'Subtotal (EUR)'] = df['Total (EUR)'] - df['Order Shipping']

In [ ]:
df[df['Billing Country Code'] == 'HR'].tail(40)

In [ ]:
# Maybe Convert HRK to EUR for other columns as well?

In [ ]:
# Fill NaN values with 0: Order Shipping
df['Order Shipping'] = df['Order Shipping'].fillna(0)

In [ ]:
# Clean the payment method column
df.loc[df['Payment Method'].str.contains('PayPal', case=False, na=False), 'Payment Method'] = 'PayPal'
df.loc[df['Payment Method'].str.contains('Stripe Safe Card', case=False, na=False), 'Payment Method'] = 'Stripe Safe Card'
df.loc[df['Payment Method'].str.contains('bank transfer', case=False, na=False), 'Payment Method'] = 'Bank Transfer'

Clean Shipping Method

In [ ]:
# Merge the 2 Shipping Method Title columns
df['Shipping Method'] = df['Shipping Method 1'].fillna(df['Shipping Method 2'])
# Drop the Shipping Method Title.1 column
df = df.drop(['Shipping Method 1', 'Shipping Method 2'], axis=1)
# Where value_counts for Shipping Method is less than 5%, replace with 'Other'
df.loc[df['Shipping Method'].isin(df['Shipping Method'].value_counts()[df['Shipping Method'].value_counts(normalize=True) < 0.02].index), 'Shipping Method'] = 'Other'

In [ ]:
# Merge 'First Name' and 'Last Name' columns into 'Full Name' column
df['Billing Full Name'] = df['Billing First Name'] + ' ' + df['Billing Last Name']

In [ ]:
# Drop 'First Name' and 'Last Name' columns
df = df.drop(['Billing First Name', 'Billing Last Name'], axis=1)

In [ ]:
# Make pie chart of shipping methods
df['Shipping Method'].value_counts().plot(kind='pie', figsize=(10, 10), autopct='%1.1f%%', startangle=90, title='Shipping Methods')


In [ ]:
# Make pie chart of payment methods
df['Payment Method'].value_counts().plot(kind='pie', figsize=(10, 10), autopct='%1.1f%%', startangle=90, title='Payment Methods')

In [ ]:
# Reorder columns
df = df[['Transaction',
         'Date',  
         'Line Items',
         'Status', 
         'Billing Full Name',
         'Billing Country Code', 
         'Shipping City', 
         'Shipping Country Code', 
         'Shipping Method', 
         'Payment Method', 
         'Stripe Fee',
         'Order Shipping', 
         'Subtotal (EUR)', 
         'Total (EUR)']]

In [ ]:
df.head()

Save rows with Status "wc-completed"

In [ ]:
# Filter only "wc-completed" orders into new dataframe
completed_orders = df[df['Status'] == "wc-completed"]

# Save the dataframe to a csv file. 
if isOriginalDataUsed:
    completed_orders.to_csv('../data/original/completed_orders.csv', index=False)
else:
    completed_orders.to_csv('../data/anonymous/completed_orders.csv', index=False)

In [ ]:
completed_orders[(completed_orders['Shipping Country Code'] == "HR") & pd.notna(completed_orders['Payment Method'])].tail(20)

In [ ]:
df_free_shipping = df
# Check if the following is true in a newly created column: above 52 EUR Cro free shipping & above 290 EUR outside Cro free shipping
df_free_shipping['Shipping Should Be Free'] = (df['Total (EUR)'] >= 52) & (df['Billing Country Code'] == "HR") | (df['Total (EUR)'] >= 290) & (df['Billing Country Code'] != "HR")
# Make a new dataframe of all rows where Free shipping should have been applied but wasn't
df_free_shipping = df_free_shipping.loc[(df['Shipping Should Be Free'] == True) & (df['Order Shipping'] > 0)]
# Save this dataframe to a csv file based on the original data used
if isOriginalDataUsed:
    df_free_shipping.to_csv('../data/original/free_shipping_not_applied_properly.csv', index=False)
else:
    df_free_shipping.to_csv('../data/anonymous/free_shipping_not_applied_properly.csv', index=False)


In [ ]:
completed_orders['Line Items'].head(20)

In [ ]:
# Compare the rest of the statuses with the "wc-completed" orders
